In [0]:
labels = ['5_o_Clock_Shadow','Arched_Eyebrows','Attractive','Bags_Under_Eyes','Bald','Bangs','Big_Lips','Big_Nose','Black_Hair',
 'Blond_Hair', 'Blurry','Brown_Hair','Bushy_Eyebrows','Chubby','Double_Chin','Eyeglasses','Goatee','Gray_Hair','Heavy_Makeup',
 'High_Cheekbones','Male','Mouth_Slightly_Open','Mustache','Narrow_Eyes','No_Beard','Oval_Face','Pale_Skin','Pointy_Nose',
 'Receding_Hairline','Rosy_Cheeks','Sideburns','Smiling','Straight_Hair','Wavy_Hair','Wearing_Earrings','Wearing_Hat',
 'Wearing_Lipstick','Wearing_Necklace','Wearing_Necktie','Young']

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls drive/My\ Drive

'Assignment 1.ipynb'		   PyTorch_multi_label_classifier.ipynb
 cartisan_assessment_exercise_2    Pytorch_textdata.ipynb
 CelebA				  'simple_dogcat_classifier (1).ipynb'
'Colab Notebooks'		   simple_dogcat_classifier.ipynb
 del-data.csv.tar.gz		   train-recognizer.ipynb
 dogcat_classifier.ipynb	   Untitled0.ipynb
 dogcat_classifier_vgg.ipynb	   Untitled1.ipynb
 download_files.ipynb		   Untitled2.ipynb
 drive_setup.ipynb		   Untitled3.ipynb
'Getting started.pdf'		   Untitled4.ipynb
 iisc_blr_task			   Untitled5.ipynb
 Image_1.zip			  'Untitled document.gdoc'
 kaggle.json			   variance_all_humidity.ipynb
 Model_40_EPCHS			   variance_all.ipynb
 python_celeba_multi-label.ipynb   variance_all_light.ipynb
 pytorch1.ipynb			   variance_all_voltage.ipynb


In [0]:
import torch
import torch.nn as nn
from torchvision import transforms
import torch.nn.functional as F
from PIL import Image
import numpy as np

In [0]:
class MultiClassifier(nn.Module):
    def __init__(self):
        super(MultiClassifier, self).__init__()
        self.ConvLayer1 = nn.Sequential(
            nn.Conv2d(3, 64, 3), # 3, 256, 256
            nn.MaxPool2d(2), # op: 16, 127, 127
            nn.ReLU(), # op: 64, 127, 127
        )
        self.ConvLayer2 = nn.Sequential(
            nn.Conv2d(64, 128, 3), # 64, 127, 127   
            nn.MaxPool2d(2), #op: 128, 63, 63
            nn.ReLU() # op: 128, 63, 63
        )
        self.ConvLayer3 = nn.Sequential(
            nn.Conv2d(128, 256, 3), # 128, 63, 63
            nn.MaxPool2d(2), #op: 256, 30, 30
            nn.ReLU() #op: 256, 30, 30
        )
        self.ConvLayer4 = nn.Sequential(
            nn.Conv2d(256, 512, 3), # 256, 30, 30
            nn.MaxPool2d(2), #op: 512, 14, 14
            nn.ReLU(), #op: 512, 14, 14
            nn.Dropout(0.2)
        )
        self.Linear1 = nn.Linear(512 * 14 * 14, 1024)
        self.Linear2 = nn.Linear(1024, 256)
        self.Linear3 = nn.Linear(256, 40)
        
        
    def forward(self, x):
        x = self.ConvLayer1(x)
        x = self.ConvLayer2(x)
        x = self.ConvLayer3(x)
        x = self.ConvLayer4(x)
        x = x.view(x.size(0), -1)
        x = self.Linear1(x)
        x = self.Linear2(x)
        x = self.Linear3(x)
        return F.sigmoid(x)

In [0]:
model_path = 'drive/My Drive/Model_40_EPCHS'

In [0]:
model = torch.load(model_path, map_location=torch.device('cpu'))

In [0]:
def get_tensor(img):
    tfms = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])
    
    return tfms(Image.open(img)).unsqueeze(0)

In [0]:
def predict(img, label_lst, model):
    tnsr = get_tensor(img)
    op = model(tnsr)
    op_b = torch.round(op)
    
    op_b_np = torch.Tensor.cpu(op_b).detach().numpy()
    
    preds = np.where(op_b_np == 1)[1]
    
    print(preds)
    
    label = []
    for i in preds:
        label.append(label_lst[i])
        
    return label, op

In [37]:
predict('/content/images.jpeg', labels, model)

[ 0  7 20 24 32 34]


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


(['5_o_Clock_Shadow',
  'Big_Nose',
  'Male',
  'No_Beard',
  'Straight_Hair',
  'Wearing_Earrings'],
 tensor([[6.2407e-01, 4.2582e-09, 2.2081e-03, 8.6636e-03, 1.7147e-07, 4.9673e-04,
          5.3532e-05, 5.7182e-01, 1.9525e-02, 1.8435e-06, 1.3335e-01, 5.6491e-07,
          4.0920e-03, 1.6351e-05, 7.5260e-04, 4.1949e-01, 9.5306e-03, 6.3661e-04,
          3.3259e-07, 1.5029e-04, 9.9932e-01, 7.8361e-02, 2.3443e-01, 4.0281e-05,
          8.8718e-01, 6.4305e-04, 1.7309e-06, 4.7559e-03, 2.4093e-03, 2.0086e-03,
          4.0016e-02, 8.7549e-07, 6.6590e-01, 1.0298e-05, 6.7072e-01, 2.9100e-04,
          2.2234e-06, 2.8025e-04, 4.1978e-07, 1.2359e-03]],
        grad_fn=<SigmoidBackward>))